In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
import datetime

%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
# Read in csv data
cats = pd.read_csv('..//data/cats.csv')
barnyard = pd.read_csv('..//data/barnyard.csv')
dogs = pd.read_csv('..//data/dogs.csv')
horses = pd.read_csv('..//data/horses.csv')
rabbits = pd.read_csv('..//data/rabbits.csv')
scales = pd.read_csv('..//data/scales.csv')
smallandfurry = pd.read_csv('..//data/smallandfurry.csv')

In [3]:
print(cats.shape)
print(barnyard.shape)
print(dogs.shape)
print(horses.shape)
print(rabbits.shape)
print(scales.shape)
print(smallandfurry.shape)

(14240, 50)
(27, 50)
(19893, 50)
(16, 62)
(161, 62)
(2, 49)
(63, 51)


In [4]:
# Combine datasets into one dataframe
df = pd.concat([cats, barnyard, dogs, horses, rabbits, scales, smallandfurry])
df.reset_index(drop=True, inplace=True)
df.shape

(34402, 63)

### Drop Duplicates

In [5]:
df = df.drop_duplicates()
df.shape

(34010, 63)

In [6]:
df.head()

,id,organization_id,url,type,species,age,gender,size,coat,tags,name,description,organization_animal_id,photos,videos,status,status_changed_at,published_at,distance,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,primary_photo_cropped.small,primary_photo_cropped.medium,primary_photo_cropped.large,primary_photo_cropped.full,contact.email,contact.phone,contact.address.address1,contact.address.address2,contact.address.city,contact.address.state,contact.address.postcode,contact.address.country,_links.self.href,_links.type.href,_links.organization.href,primary_photo_cropped,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,id.1
0,70208019.0,TN801,https://www.petfinder.com/cat/madame-kitty-702...,Cat,Cat,Young,Female,Medium,Short,[],Madame Kitty,"Meet Madame Kitty, a 2 year old DSH cat. She i...",NaN,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,[],adopted,2024-01-22T13:31:21+0000,2023-12-31T23:19:08+0000,120.4729,Domestic Short Hair,NaN,False,False,Calico,NaN,NaN,True,True,0.0,False,True,True,NaN,NaN,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,meowmeowkittyrescue@yahoo.com,NaN,"2288 Gunbarrel Rd #172, Chattanooga, TN 37421",NaN,Chattanooga,TN,37421.0,US,/v2/animals/70208019,/v2/types/cat,/v2/organizations/tn801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,70207823.0,TN597,https://www.petfinder.com/cat/truffles-7020782...,Cat,Cat,Baby,Female,Small,Short,"['Friendly', 'Affectionate', 'Playful']",Truffles,Truffles is the new kid on the block. This pr...,NaN,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,[],adopted,2024-01-18T00:05:29+0000,2023-12-31T22:40:37+0000,33.2060,Dilute Calico,NaN,True,False,Dilute Calico,Brown / Chocolate,NaN,True,True,0.0,False,True,True,True,True,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,NaN,(931) 623-1266,1163 Blake Road,NaN,Lyles,TN,37098.0,US,/v2/animals/70207823,/v2/types/cat,/v2/organizations/tn597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70207450.0,TN369,https://www.petfinder.com/cat/teetee-70207450/...,Cat,Cat,Baby,Male,Small,Short,"['Friendly', 'Affectionate', 'Playful']",TeeTee,Meet TeeTee! He is an extremely adorable and s...,1207-23,[{'small': 'https://dl5zpyw5k3jeb.cloudfront.n...,[],adopted,2024-01-06T00:56:45+0000,2023-12-31T21:45:47+0000,7.6471,Domestic Short Hair,NaN,False,False,Tabby (Gray / Blue / Silver),NaN,NaN,True,True,0.0,False,True,NaN,True,True,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,fosterkittenstn@gmail.com,NaN,P.O. Box 140898,NaN,Nashville,TN,37214.0,US,/v2/animals/70207450,/v2/types/cat,/v2/organizations/tn369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,70203376.0,TN90,https://www.petfinder.com/cat/ac-25055-7020337...,Cat,Cat,Young,Male,Medium,NaN,[],AC 25055,NaN,A2023C688,[],[],adopted,2023-12-31T10:03:59+0000,2023-12-31T10:03:58+0000,223.6873,Domestic Short Hair,NaN,False,False,NaN,NaN,NaN,False,False,0.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(423) 639-4771,400 N Rufe Taylor Rd,NaN,Greeneville,TN,37745.0,US,/v2/animals/70203376,/v2/types/cat,/v2/organizations/tn90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,70203375.0,TN90,https://www.petfinder.com/cat/ac-25035-7020337...,Cat,Cat,Young,Female,Medium,NaN,[],AC 25035,NaN

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34010 entries, 0 to 34401
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            34009 non-null  float64
 1   organization_id               34009 non-null  object 
 2   url                           34009 non-null  object 
 3   type                          34009 non-null  object 
 4   species                       34009 non-null  object 
 5   age                           34009 non-null  object 
 6   gender                        34009 non-null  object 
 7   size                          34009 non-null  object 
 8   coat                          13600 non-null  object 
 9   tags                          34009 non-null  object 
 10  name                          34009 non-null  object 
 11  description                   19496 non-null  object 
 12  organization_animal_id        18989 non-null  object 
 13  photos

In [8]:
# Drop weird row with all nulls
df[df['breeds.primary'].isna()]
df = df.drop(33944)
df.shape

(34009, 63)

### Clean dtypes

In [9]:
#df.info()

In [10]:
# Change 'published_at' and 'status_changed'_at columns to datetime 
df['published_at'] = pd.to_datetime(df['published_at'])
df['status_changed_at'] = pd.to_datetime(df['status_changed_at'])

In [11]:
# Define function to switch dtypes to string
def string_clean(column):
    x = df[column].astype('string')
    return x

In [12]:
# Test
df['organization_id'] = string_clean('organization_id')

In [13]:
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = string_clean(column)

In [14]:
#df.info()

### Drop unnecessary columns

In [15]:
#print(df.columns)

In [16]:
drop_list = [
    'organization_animal_id',
    'photos', 
    'videos',
    'distance',
    'primary_photo_cropped.small', 
    'primary_photo_cropped.medium',
    'primary_photo_cropped.large',
    'primary_photo_cropped.full',
    'contact.email', 
    'contact.phone', 
    'contact.address.address1',
    'contact.address.address2',
    'contact.address.country', 
    '_links.self.href', 
    '_links.type.href',
    '_links.organization.href',
    'primary_photo_cropped',
    'Unnamed: 50',
    'Unnamed: 51', 
    'Unnamed: 52', 
    'Unnamed: 53', 
    'Unnamed: 54',
    'Unnamed: 55', 
    'Unnamed: 56', 
    'Unnamed: 57', 
    'Unnamed: 58',
    'Unnamed: 59', 
    'Unnamed: 60', 
    'Unnamed: 61', 
    'id.1'
]

In [17]:
df.drop(columns=drop_list, inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34009 entries, 0 to 34401
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   id                          34008 non-null  float64            
 1   organization_id             34008 non-null  string             
 2   url                         34008 non-null  string             
 3   type                        34008 non-null  string             
 4   species                     34008 non-null  string             
 5   age                         34008 non-null  string             
 6   gender                      34008 non-null  string             
 7   size                        34008 non-null  string             
 8   coat                        13599 non-null  string             
 9   tags                        34008 non-null  string             
 10  name                        34008 non-null  string             

### Clean name strings

In [19]:
df['name'] = df['name'].str.strip()  
df['name'] = df['name'].str.title()

In [20]:
# Check 
#df['name'].value_counts().head(50)

### Reformat date column 

In [21]:
# Change published_date format from ISO to normal datetime format
df['published_date'] = df['published_at'].dt.strftime('%m/%d/%y')

In [22]:
# Change status_change_ date format from ISO to normal datetime format
df['status_change_date'] = df['status_changed_at'].dt.strftime('%m/%d/%y')

### Add column for time between published date and status change dates

In [23]:
df['adoption_time'] = (df['status_changed_at'] - df['published_at']).dt.days

### Add month, day, year separate columns 

In [24]:
# Published at
df['month_published'] = df['published_at'].dt.month
df['day_published'] = df['published_at'].dt.day
df['year_published'] = df['published_at'].dt.year
df['time_published'] = df['published_at'].dt.time
# Status changed at
df['month_status_change'] = df['status_changed_at'].dt.month
df['day_status_change'] = df['status_changed_at'].dt.day
df['year_status_change'] = df['status_changed_at'].dt.year
df['time_status_change'] = df['status_changed_at'].dt.time

In [25]:
# Check
df[['status_change_date', 'published_date', 'adoption_time', 'month_published', 'day_published', 'time_published', 'time_status_change']].head()

,status_change_date,published_date,adoption_time,month_published,day_published,time_published,time_status_change
0,01/22/24,12/31/23,21.0,12.0,31.0,23:19:08,13:31:21
1,01/18/24,12/31/23,17.0,12.0,31.0,22:40:37,00:05:29
2,01/06/24,12/31/23,5.0,12.0,31.0,21:45:47,00:56:45
3,12/31/23,12/31/23,0.0,12.0,31.0,10:03:58,10:03:59
4,12/31/23,12/31/23,0.0,12.0,31.0,10:03:56,10:03:57


In [26]:
#df.columns

### Add adoption_time column

In [27]:
# Convert to datetime format
df['published_date'] = pd.to_datetime(df['published_date'], format='%m/%d/%y')
df['status_change_date'] = pd.to_datetime(df['status_change_date'], format='%m/%d/%y')

# Calculate adoption time (time available for adoption)
df['adoption_time'] = (df['status_change_date'] - df['published_date']).dt.days

In [28]:
# Check
df.head(1)

,id,organization_id,url,type,species,age,gender,size,coat,tags,name,description,status,status_changed_at,published_at,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,contact.address.city,contact.address.state,contact.address.postcode,published_date,status_change_date,adoption_time,month_published,day_published,year_published,time_published,month_status_change,day_status_change,year_status_change,time_status_change
0,70208019.0,TN801,https://www.petfinder.com/cat/madame-kitty-702...,Cat,Cat,Young,Female,Medium,Short,[],Madame Kitty,"Meet Madame Kitty, a 2 year old DSH cat. She i...",adopted,2024-01-22 13:31:21+00:00,2023-12-31 23:19:08+00:00,Domestic Short Hair,<NA>,False,False,Calico,<NA>,<NA>,True,True,0.0,False,True,True,<NA>,<NA>,Chattanooga,TN,37421.0,2023-12-31,2024-01-22,22.0,12.0,31.0,2023.0,23:19:08,1.0,22.0,2024.0,13:31:21


### Verify the published vs change dates

In [29]:
max_pub = df['published_at'].max()
min_pub = df['published_at'].min()

max_schange = df['status_changed_at'].max()
min_schange = df['status_changed_at'].min()

print('earliest publish date: ', min_pub)
print('latest publish date: ', max_pub)
print('earliest status change date: ', min_schange)
print('latest status change date: ', max_schange)

earliest publish date:  2023-01-01 00:15:33+00:00
latest publish date:  2023-12-31 23:19:08+00:00
earliest status change date:  2023-01-01 05:15:33+00:00
latest status change date:  2024-02-08 17:28:30+00:00


### Clean tags

In [30]:
df['tags'] = df['tags'].str.strip('[]')

In [31]:
df['tags'] = df['tags'].str.replace("'", "")

In [32]:
#df.groupby('tags')['id'].count().reset_index(name='count').sort_values('count', ascending=False).head(50)

In [33]:
df['tags'] = df['tags'].str.strip()  
df['tags'] = df['tags'].str.title()  

In [34]:
df.tags

0                                                         
1                          Friendly, Affectionate, Playful
2                          Friendly, Affectionate, Playful
3                                                         
4                                                         
                               ...                        
34397                                                     
34398                             Shy, Confident, Friendly
34399                            Playful, Social, Friendly
34400                      Playful, Friendly, Loves Treats
34401    Loyal, Gentle, Smart, Curious, Independent, Fr...
Name: tags, Length: 34009, dtype: string

### Clean Cities

In [35]:
df['contact.address.city'] = df['contact.address.city'].str.strip().str.title()

In [36]:
df.groupby('contact.address.city')['id'].count().reset_index(name='count').sort_values('count', ascending=False).head()

,contact.address.city,count
60,Greeneville,10371
132,Shelbyville,2098
105,Nashville,2027
34,Crossville,2021
130,Sevierville,874


In [37]:
tn_cities = df['contact.address.city'].nunique()
print('Number of Cities:', tn_cities)

Number of Cities: 156


In [38]:
df.groupby('contact.address.city')['id'].count().head(50)

contact.address.city
Afton               1
Alcoa               5
Andersonville      63
Antioch           105
Arlington           9
Ashland City        5
Athens             77
Atoka              12
Bartlett          157
Big Rock           19
Blountville       783
Bolivar            65
Brentwood         203
Bristol           163
Brownsville        88
Buchanan            7
Camden             39
Caryville          29
Centerville        25
Chapel Hill        11
Chapmansboro        1
Chattanooga       860
Chuckey            23
Church Hill         4
Clarksville       134
Cleveland          67
Clinton             3
College Grove      10
Collierville      348
Columbia          150
Cookeville        112
Cordova            27
Cosby             160
Covington          93
Crossville       2021
Dandridge         418
Dayton             30
Dickson            35
Donelson            8
Dover              63
Dowelltown         29
Duck River         70
Dyer                1
Dyersburg         351
Eagleville 

### Clean breed strings

In [39]:
df['breeds.primary'] = df['breeds.primary'].str.strip().str.title()

In [40]:
df['breeds.secondary'] = df['breeds.secondary'].str.strip().str.title()

In [41]:
df.groupby('breeds.secondary')['id'].count().reset_index(name='count').sort_values('count', ascending=True).tail()

,breeds.secondary,count
74,Domestic Short Hair,371
136,Mixed Breed,476
110,Hound,530
195,Terrier,658
122,Labrador Retriever,661


### Clean color strings

In [42]:
#df.columns

In [43]:
df['colors.primary'] = df['colors.primary'].str.strip().str.title()
df['colors.secondary'] = df['colors.secondary'].str.strip().str.title()
df['colors.tertiary'] = df['colors.tertiary'].str.strip().str.title()

In [44]:
df.groupby(['colors.primary', 'colors.secondary', 'colors.tertiary'])['id'].count().reset_index(name='count').sort_values('count', ascending=False)

,colors.primary,colors.secondary,colors.tertiary,count
49,Black,Brown / Chocolate,Bicolor,28
55,Black,Brown / Chocolate,White / Cream,24
47,Black,Brindle,White / Cream,13
89,Black,Yellow / Tan / Blond / Fawn,White / Cream,12
361,Yellow / Tan / Blond / Fawn,Black,White / Cream,12
...,...,...,...,...
159,Brown / Chocolate,Yellow / Tan / Blond / Fawn,Gray / Blue / Silver,1
158,Brown / Chocolate,Yellow / Tan / Blond / Fawn,Black,1
157,Brown / Chocolate,White / Cream,Yellow / Tan / Blond / Fawn,1
156,Brown / Chocolate,White / Cream,"Tricolor (Brown, Black, & White)",1


In [45]:
primary = df['colors.primary'].nunique()
secondary = df['colors.secondary'].nunique()
tertiary = df['colors.tertiary'].nunique()

print('primary colors: ', primary)
print('secondary colors: ', secondary)
print('tertiary colors: ', tertiary)

primary colors:  57
secondary colors:  55
tertiary colors:  47


## Clean breeds.mixed column 
#### Set all False Values for 'Mixed Breed' in primary column to False

In [46]:
df.groupby('breeds.mixed')['id'].count()

breeds.mixed
False    17465
True     16543
Name: id, dtype: int64

In [47]:
df[df['breeds.primary'] == 'Mixed Breed'].groupby('breeds.mixed')['id'].count()

breeds.mixed
False    809
True     435
Name: id, dtype: int64

In [48]:
breeds_mixed = df[df['breeds.primary'] == 'Mixed Breed']
breeds_mixed.groupby('breeds.mixed')['id'].count()
breeds_mixed[breeds_mixed['breeds.mixed'] == 'False']

,id,organization_id,url,type,species,age,gender,size,coat,tags,name,description,status,status_changed_at,published_at,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,contact.address.city,contact.address.state,contact.address.postcode,published_date,status_change_date,adoption_time,month_published,day_published,year_published,time_published,month_status_change,day_status_change,year_status_change,time_status_change
14324,70194352.0,TN927,https://www.petfinder.com/dog/tuff-70194352/tn...,Dog,Dog,Young,Male,Large,<NA>,,Tuff,<NA>,adopted,2024-01-10 10:08:21+00:00,2023-12-30 10:13:33+00:00,Mixed Breed,<NA>,False,False,<NA>,<NA>,<NA>,False,False,<NA>,False,False,<NA>,<NA>,<NA>,Pulaski,TN,38478.0,2023-12-30,2024-01-10,11.0,12.0,30.0,2023.0,10:13:33,1.0,10.0,2024.0,10:08:21
14325,70194353.0,TN927,https://www.petfinder.com/dog/blu-70194353/tn/...,Dog,Dog,Young,Male,Large,<NA>,,Blu,<NA>,adopted,2024-02-04 10:12:26+00:00,2023-12-30 10:13:33+00:00,Mixed Breed,<NA>,False,False,<NA>,<NA>,<NA>,True,False,<NA>,False,False,True,True,<NA>,Pulaski,TN,38478.0,2023-12-30,2024-02-04,36.0,12.0,30.0,2023.0,10:13:33,2.0,4.0,2024.0,10:12:26
14512,70111595.0,TN97,https://www.petfinder.com/dog/travis-70111595/...,Dog,Dog,Baby,Male,Large,Short,,Travis,Available Saturday 12/23,adopted,2023-12-29 18:36:15+00:00,2023-12-21 04:03:31+00:00,Mixed Breed,<NA>,False,False,<NA>,<NA>,<NA>,True,False,<NA>,False,True,<NA>,<NA>,<NA>,Sevierville,TN,37862.0,2023-12-21,2023-12-29,8.0,12.0,21.0,2023.0,04:03:31,12.0,29.0,2023.0,18:36:15
14537,70099173.0,MS108,https://www.petfinder.com/dog/dillion-70099173...,Dog,Dog,Young,Male,Small,Short,"Friendly, Affectionate, Loyal, Gentle, Playful...",Dillion,Duke is about 6 mos old and just the sweetest ...,adopted,2024-01-03 17:43:55+00:00,2023-12-20 04:09:36+00:00,Mixed Breed,<NA>,False,False,Black,<NA>,<NA>,True,True,<NA>,False,True,True,True,True,Memphis,TN,38103.0,2023-12-20,2024-01-03,14.0,12.0,20.0,2023.0,04:09:36,1.0,3.0,2024.0,17:43:55
14661,70057776.0,TN927,https://www.petfinder.com/dog/hippie-70057776/...,Dog,Dog,Adult,Female,Large,<NA>,,Hippie,<NA>,adopted,2024-01-18 10:03:49+00:00,2023-12-16 10:01:48+00:00,Mixed Breed,<NA>,False,False,<NA>,<NA>,<NA>,False,False,<NA>,False,False,<NA>,<NA>,<NA>,Pulaski,TN,38478.0,2023-12-16,2024-01-18,33.0,12.0,16.0,2023.0,10:01:48,1.0,18.0,2024.0,10:03:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34041,60481174.0,TN1019,https://www.petfinder.com/dog/charlotte-604811...,Dog,Dog,Young,Female,Large,<NA>,,Charlotte,"If you are interested in adopting me, please c...",adoptable,2023-03-05 19:01:26+00:00,2023-03-05 19:01:19+00:00,Mixed Breed,<NA>,False,False,<NA>,<NA>,<NA>,False,False,<NA>,False,True,<NA>,<NA>,<NA>,Lawrenceburg,TN,38464.0,2023-03-05,2023-03-05,0.0,3.0,5.0,2023.0,19:01:19,3.0,5.0,2023.0,19:01:26
34042,60481186.0,TN1019,https://www.petfinder.com/dog/lucky-60481186/t...,Dog,Dog,Young,Male,Large,<NA>,,Lucky,"If you are interested in adopting me, please c...",adoptable,2023-03-05 19:01:22+00:00,2023-03-05 19:01:19+00:00,Mixed Breed,<NA>,False,False,<NA>,<NA>,<NA>,False,False,<NA>,False,True,<NA>,<NA>,<NA>,Lawrenceburg,TN,38464.0,2023-03-05,2023-03-05,0.0,3.0,5.0,2023.0,19:01:19,3.0,5.0,2023.0,19:01:22
34090,59948067.0,TN26,https://www.petfinder.com/dog/boo-59948067/tn/...,Dog,Dog,Young,Female,Medium,Short,"Friendly, Affectionate, Playful, Funny, Loves ...",Boo,Very sweet girl! Loves to cuddle and play!,adoptable,2023-02-14 17:05:07+00:00,2023-02-14 17:05:06+00:00,Mixed Breed,<NA>,False,False,<NA>,<NA>,<NA>,True,False,<NA>,False,True,<NA>,<NA>,<NA>,Shelbyville,TN,37160.0,2023-02-14,2023-02-14,0.0,2.0,14.0,2023.0,17:05

In [49]:
# Replace False with True
df.loc[df['breeds.primary'] == 'Mixed Breed', 'breeds.mixed'] = 'True'

In [50]:
# Check
breeds_mixed = df[df['breeds.primary'] == 'Mixed Breed']
breeds_mixed[['breeds.primary', 'breeds.mixed']]
#breeds_mixed.groupby('breeds.mixed')['id'].count()
#breeds_mixed[breeds_mixed['breeds.mixed'] == 'False']

,breeds.primary,breeds.mixed
14324,Mixed Breed,True
14325,Mixed Breed,True
14401,Mixed Breed,True
14402,Mixed Breed,True
14404,Mixed Breed,True
...,...,...
34094,Mixed Breed,True
34105,Mixed Breed,True
34130,Mixed Breed,True
34136,Mixed Breed,True


In [51]:
### Switch breeds secondary to breeds primary and replace secondary with null values

In [52]:
breeds_mixed.groupby(['breeds.primary', 'breeds.secondary', 'breeds.mixed'])['id'].count().reset_index(name='count')

,breeds.primary,breeds.secondary,breeds.mixed,count
0,Mixed Breed,Affenpinscher,True,1
1,Mixed Breed,American Staffordshire Terrier,True,6
2,Mixed Breed,Australian Cattle Dog / Blue Heeler,True,1
3,Mixed Breed,Basset Hound,True,5
4,Mixed Breed,Beagle,True,4
5,Mixed Breed,Black Labrador Retriever,True,8
6,Mixed Breed,Boxer,True,1
7,Mixed Breed,Carolina Dog,True,1
8,Mixed Breed,Chihuahua,True,1
9,Mixed Breed,Dachshund,True,1


In [53]:
# Replace 'Mixed Breed' with Value in the breeds.secondary column

df.loc[df['breeds.primary'] == 'Mixed Breed', 'breeds.primary'] = df.loc[df['breeds.primary'] == 'Mixed Breed', 'breeds.secondary']

# Replace 'Mixed Breed' in breeds.secondary with 'Unknown'
df.loc[df['breeds.secondary'] == 'Mixed Breed', 'breeds.secondary'] = 'Unknown'

In [54]:
# Check
breeds_mixed = df[df['breeds.primary'] == 'Mixed Breed']
breeds_mixed.groupby(['breeds.primary', 'breeds.secondary', 'breeds.mixed'])['id'].count().reset_index(name='count')

,breeds.primary,breeds.secondary,breeds.mixed,count


In [55]:
df[df['breeds.secondary'] == 'Mixed Breed']

,id,organization_id,url,type,species,age,gender,size,coat,tags,name,description,status,status_changed_at,published_at,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,contact.address.city,contact.address.state,contact.address.postcode,published_date,status_change_date,adoption_time,month_published,day_published,year_published,time_published,month_status_change,day_status_change,year_status_change,time_status_change


In [56]:
# Check
df[df['breeds.secondary'] == 'Mixed Breed']

,id,organization_id,url,type,species,age,gender,size,coat,tags,name,description,status,status_changed_at,published_at,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,colors.primary,colors.secondary,colors.tertiary,attributes.spayed_neutered,attributes.house_trained,attributes.declawed,attributes.special_needs,attributes.shots_current,environment.children,environment.dogs,environment.cats,contact.address.city,contact.address.state,contact.address.postcode,published_date,status_change_date,adoption_time,month_published,day_published,year_published,time_published,month_status_change,day_status_change,year_status_change,time_status_change


In [57]:
mixed = df[df['breeds.mixed'] == 'True']
mixed = mixed.groupby(['breeds.primary', 'breeds.secondary', 'breeds.mixed'])['id'].count().reset_index(name='count')
mixed.to_csv('..//data/mixed_breeds.csv')

### Import to csv for data analysis

In [59]:
df.groupby('species')['id'].count()

species
Cat                14116
Dog                19676
Goat                   4
Guinea Pig            57
Hamster                2
Hedgehog               2
Horse                 12
Miniature Horse        3
Mule                   1
Pig                    5
Pot Bellied           18
Rabbit               108
Rat                    2
Reptile                1
Snake                  1
Name: id, dtype: int64

In [62]:
df.columns

Index(['id', 'organization_id', 'url', 'type', 'species', 'age', 'gender',
       'size', 'coat', 'tags', 'name', 'description', 'status',
       'status_changed_at', 'published_at', 'breeds.primary',
       'breeds.secondary', 'breeds.mixed', 'breeds.unknown', 'colors.primary',
       'colors.secondary', 'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'contact.address.city', 'contact.address.state',
       'contact.address.postcode', 'published_date', 'status_change_date',
       'adoption_time', 'month_published', 'day_published', 'year_published',
       'time_published', 'month_status_change', 'day_status_change',
       'year_status_change', 'time_status_change'],
      dtype='object')

In [64]:
df.groupby(['year_published', 'year_status_change', 'status'])['id'].count()

year_published  year_status_change  status   
2023.0          2023.0              adoptable     3022
                                    adopted      29735
                2024.0              adoptable      105
                                    adopted       1146
Name: id, dtype: int64

In [65]:
df.shape

(34009, 44)

In [66]:
# Remove animals who were adopted in 2024 from dataframe
df = df[df['year_status_change'] != 2024]

In [69]:
# Check
print(df.shape)
df.groupby(['year_published', 'year_status_change', 'status'])['id'].count()

(32758, 44)


year_published  year_status_change  status   
2023.0          2023.0              adoptable     3022
                                    adopted      29735
Name: id, dtype: int64

In [72]:
df.columns

Index(['id', 'organization_id', 'url', 'type', 'species', 'age', 'gender',
       'size', 'coat', 'tags', 'name', 'description', 'status',
       'status_changed_at', 'published_at', 'breeds.primary',
       'breeds.secondary', 'breeds.mixed', 'breeds.unknown', 'colors.primary',
       'colors.secondary', 'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'contact.address.city', 'contact.address.state',
       'contact.address.postcode', 'published_date', 'status_change_date',
       'adoption_time', 'month_published', 'day_published', 'year_published',
       'time_published', 'month_status_change', 'day_status_change',
       'year_status_change', 'time_status_change'],
      dtype='object')

In [71]:
# Export to CSV
df.to_csv('../data/cleaned_pets.csv', index=False)